In [1]:
import pandas as pd 
import numpy as np
from utils import *
import time

In [2]:
df = pd.read_csv("./data/CONTEST_DATA_IN_SAMPLE_1.csv",header=None)

In [3]:
df.columns = ['time','stkcd','open','high','low','close','volume']

In [4]:
close_df = pd.DataFrame()
high_df = pd.DataFrame()
low_df = pd.DataFrame()
volume_df = pd.DataFrame()
for sub_df in df.groupby('stkcd'):
    close_df[sub_df[0]] = sub_df[1].reset_index().close
    high_df[sub_df[0]] = sub_df[1].reset_index().high
    low_df[sub_df[0]] = sub_df[1].reset_index().low
    volume_df[sub_df[0]] = sub_df[1].reset_index().volume
highs = high_df.values
lows = low_df.values
closes = close_df.values
volumes = volume_df.values

rts = np.diff(closes,axis=0)/closes[:-1]

In [10]:
rts = np.diff(closes,axis=0)/closes[:-1]

In [12]:
moms = []
vols = []
max52s = []
min52s = []
ccis = []
Ks = []
Ds = []
Js = []
rsis = []
trixs = []
willrs = []
macds = []
natrs = []

for i in range(100,1100):
    c = closes[:i]
    h = highs[:i]
    l = lows[:i]

    mom=get_mom(c,3)
    vol=get_vol(c,3)
    max52=get_52weekhigh(c)
    min52=get_52weeklow(c)
    cci = get_cci(h,l,c)
    K,D,J = get_kdj(h,l,c)
    rsi = get_rsi(c)
    trix = get_trix(c)
    willr = get_willr(h,l,c)
    macd = get_macd(c)
    natr = get_natr(h,l,c)

    moms.append(mom)
    vols.append(vol)
    max52s.append(max52)
    min52s.append(min52)
    ccis.append(cci)
    Ks.append(K)
    Ds.append(D)
    Js.append(J)
    rsis.append(rsi)
    trixs.append(trix)
    willrs.append(willr)
    macds.append(macd)
    natrs.append(natr)